In [2]:
!pip install optuna
!wget https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/dataset.csv

     |████████████████████████████████| 308 kB 7.0 MB/s 
     |████████████████████████████████| 209 kB 15.4 MB/s 
     |████████████████████████████████| 80 kB 5.9 MB/s 
     |████████████████████████████████| 75 kB 3.5 MB/s 
     |████████████████████████████████| 112 kB 14.0 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 149 kB 18.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=a86f51aa398c54801eee640ea62998ebfd27f302ae2b558da52971c6b9f48aee
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
--2021-11-29 22:38:58--  https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/dataset.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubuserco

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from sklearn.model_selection import cross_validate

class Objective:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __call__(self, trial):
        params = {
            'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
            'C': trial.suggest_loguniform('C', 0.0001, 10),
            'max_iter': trial.suggest_int('max_iter', 100, 100000)
        }
        model = LogisticRegression(**params)
        scores = cross_validate(model,
                                            X=self.X,
                                            y=self.y,
                                            scoring='accuracy',
                                            n_jobs=-1)
        return scores['test_score'].mean()

training_data = np.genfromtxt('dataset.csv', delimiter=',', dtype=np.int32)
X = training_data[:,:-1]
y = training_data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=101
)

objective = Objective(X_train, y_train)
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=60)
print('params:', study.best_params)

[I 2021-11-29 22:52:41,284] A new study created in memory with name: no-name-b3497a3f-4f78-4bd6-b37f-92a4ec19a378
[I 2021-11-29 22:52:42,534] Trial 0 finished with value: 0.9198330856454422 and parameters: {'solver': 'liblinear', 'C': 0.002340087617843422, 'max_iter': 60772}. Best is trial 0 with value: 0.9198330856454422.
[I 2021-11-29 22:52:42,895] Trial 1 finished with value: 0.9214159647422043 and parameters: {'solver': 'newton-cg', 'C': 0.0028523671057923985, 'max_iter': 26131}. Best is trial 1 with value: 0.9214159647422043.
[I 2021-11-29 22:52:43,195] Trial 2 finished with value: 0.9225467388329424 and parameters: {'solver': 'sag', 'C': 0.0033688169952285733, 'max_iter': 7683}. Best is trial 2 with value: 0.9225467388329424.
[I 2021-11-29 22:52:43,535] Trial 3 finished with value: 0.9289924005432934 and parameters: {'solver': 'liblinear', 'C': 0.2619828846525913, 'max_iter': 44422}. Best is trial 3 with value: 0.9289924005432934.
[I 2021-11-29 22:52:43,933] Trial 4 finished with

params: {'solver': 'newton-cg', 'C': 0.09462162204832922, 'max_iter': 73096}
